In [1]:
#The data contains a combination of strings and numbers. We can not leave our target labels in the current 
# string format. This is because TensorFlow does not understand strings as labels.
#https://biolib.com/shd2020/Diabetes-Machine-Learning-Data/

#end notes : the pipeline work but the model is not correct. Need to find a better model that fits.

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Load Data

In [3]:
real_data = pd.read_csv('data/real_data.csv')
synthetic_data = pd.read_csv('data/synthetic_data.csv')

In [4]:
synthetic_data['readmitted'].unique()

array(['<30', 'NO', '>30'], dtype=object)

In [5]:
#define function for classifying players based on points
def f(row):
    if row['readmitted'] == '<30':
        val = 1
    elif row['readmitted'] == '>30':
        val = 2
    else:
        val = 3
    return val

#create new column 'Good' using the function above
synthetic_data['readmitted_num'] = synthetic_data.apply(f, axis=1)

In [6]:
dataframe = synthetic_data.copy()
dataframe.head()

,race,gender,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,_diag_1,_diag_2,_diag_3,readmitted_num
0,Caucasian,Female,[60-70),3,40,0,14,0,1,1,...,No,No,No,No,Yes,<30,Supplemental classification,Diseases of the skin and subcutaneous tissue,"Endocrine, nutritional, and metabolic diseases...",1
1,Caucasian,Male,[50-60),5,40,0,5,0,0,0,...,No,No,No,No,Yes,NO,Injury and poisoning,Diabetes mellitus,Diseases of the circulatory system,3
2,Caucasian,Female,[50-60),1,43,0,23,0,0,3,...,No,No,No,Ch,Yes,>30,Diseases of the circulatory system,Diseases of the skin and subcutaneous tissue,Injury and poisoning,2
3,Caucasian,Female,[70-80),1,41,5,14,0,12,3,...,No,No,No,Ch,Yes,>30,Diseases of the respiratory system,Diseases of the musculoskeletal system and con...,Diseases of the circulatory system,2
4,Caucasian,Female,[80-90),4,68,2,17,0,0,1,...,No,No,No,Ch,Yes,>30,Diseases of the circulatory system,Diseases of the respiratory system,Injury and poisoning,2


# Split the dataframe into train, validation, and test

In [7]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

50201 train examples
12551 validation examples
15689 test examples


# Using AutoML

In [8]:
import autokeras as ak
from autokeras import StructuredDataClassifier


In [9]:
train = train.drop(columns='readmitted')

x_train = train.drop(columns='readmitted_num')
y_train = train['readmitted_num']

input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=True)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node, 
    overwrite=True,
    max_trials=3)
clf.fit(x_train, y_train, epochs=10)

Trial 3 Complete [00h 00m 47s]
val_loss: 0.8106391429901123

Best val_loss So Far: 0.7129804491996765
Total elapsed time: 00h 02m 19s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
1569/1569 [==============================] - 2s 1ms/step - loss: 0.8915 - accuracy: 0.6145
Epoch 2/10
1569/1569 [==============================] - 2s 1ms/step - loss: 0.8039 - accuracy: 0.6575
Epoch 3/10
1569/1569 [==============================] - 2s 1ms/step - loss: 0.7747 - accuracy: 0.6683
Epoch 4/10
1569/1569 [==============================] - 2s 1ms/step - loss: 0.7535 - accuracy: 0.6785
Epoch 5/10
1569/1569 [==============================] - 2s 1ms/step - loss: 0.7414 - accuracy: 0.6836
Epoch 6/10
1569/1569 [==============================] - 2s 2ms/step - loss: 0.7322 - accuracy: 0.6885
Epoch 7/10
1569/1569 [==============================] - 2s 1ms/step - loss: 0.7256 - accuracy: 0.6917
Epoch 8/10
1569/1569 [==============================] - 2s 1ms/step - loss: 0.7207 - accuracy: 0.6932
Epoch 9/10
1

In [10]:
test = test.drop(columns='readmitted')

x_test = test.drop(columns='readmitted_num')
y_test = test['readmitted_num']

predicted_y = clf.predict(x_test)
# Evaluate the best model with testing data.
print(clf.evaluate(x_test,y_test))

491/491 [==============================] - 1s 1ms/step - loss: 0.7164 - accuracy: 0.6914
[0.7164263129234314, 0.6914398670196533]


In [11]:
model = clf.export_model()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 41)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 41)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                1344      
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                

In [1]:
real_data['readmitted_num'] = real_data.apply(f, axis=1)
x_real_data = real_data.drop(columns='readmitted_num')
y_real_data = real_data['readmitted_num']


print(clf.evaluate(x_test,y_test))

NameError: name 'real_data' is not defined